In [1]:
import sys, os
project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

import torch
from torch import Tensor, nn
from IPDL import MatrixEstimator, ClassificationInformationPlane, AutoEncoderInformationPlane
from IPDL.optim import AligmentOptimizer, SilvermanOptimizer

import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Lambda
from torch.nn.functional import one_hot
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [83]:
from IPDL import TensorKernel

def RBF(x: Tensor, sigma: float) -> Tensor:
    '''
        Tensor Based Radial Basis Function (RBF) Kernel

        @param x: Tensor shape (n, features) or (batch, n, features)
        @param sigma
    '''
    assert x.ndim > 1 and x.ndim < 4, "The dimension of X must be 2 or 3"
    pairwise_difference = (torch.unsqueeze(x,x.ndim-1) - torch.unsqueeze(x,x.ndim-2))**2
    distance = torch.sum(pairwise_difference, dim=x.ndim)
    return torch.exp(-distance / (2*(sigma**2)) )

In [84]:
sigma = 5
a = torch.rand(4, 32, 128).cuda()

In [85]:
rbf_result = RBF(a, sigma)
A = rbf_result / rbf_result.size(-1)

In [89]:
print("Test RBF")
for i in range(len(a)):
    _result = TensorKernel.RBF(a[i], sigma)
    print(torch.all(torch.isclose(_result, rbf_result[i])))

print("Test A")
for i in range(len(a)):
    _result = TensorKernel.RBF(a[i], sigma) 
    _A = _result / _result.size(-1)
    print(torch.all(torch.isclose(_A, A[i])))

Test RBF
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
Test A
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


In [117]:
from IPDL import MatrixBasedRenyisEntropy as MRE

def entropy(A: Tensor) -> float:
    eigval, _ = torch.linalg.eigh(A)        
    epsilon = 1e-8
    eigval = eigval.abs() + epsilon 
    return -torch.sum(eigval*(torch.log2(eigval)), dim=eigval.ndim-1)

In [119]:
entropy_result = entropy(A)
_entropy_result = []
for i in range(len(a)):
    _entropy_result.append(MRE.entropy(A[i]))

print(torch.isclose(torch.hstack(_entropy_result), entropy_result))

tensor([True, True, True, True], device='cuda:0')


In [121]:
print(torch.hstack(_entropy_result))
print(entropy_result)

tensor([2.5126, 2.5617, 2.5521, 2.5437], device='cuda:0')
tensor([2.5126, 2.5617, 2.5521, 2.5437], device='cuda:0')


In [109]:
eigval, _ = torch.linalg.eigh(A)    
epsilon = 1e-8
eigval = eigval.abs() + epsilon 

entropy = -torch.sum(eigval*(torch.log2(eigval)), dim=1)
print(entropy)

tensor([2.5126, 2.5617, 2.5521, 2.5437], device='cuda:0')


In [110]:
_eigval, _ = torch.linalg.eigh(_A)
epsilon = 1e-8
_eigval = _eigval.abs() + epsilon 

print(_eigval.shape)
print(-torch.sum(_eigval*(torch.log2(_eigval))))

torch.Size([32])
tensor(2.5437, device='cuda:0')


In [111]:
_entropy = []
for i in range(len(a)):
    _result = TensorKernel.RBF(a[i], sigma) 
    _A = _result / _result.size(-1)

    _eigval, _ = torch.linalg.eigh(_A)
    epsilon = 1e-8
    _eigval = _eigval.abs() + epsilon 

    _entropy.append(-torch.sum(_eigval*(torch.log2(_eigval)))) 

print(torch.isclose(torch.hstack(_entropy), entropy))

tensor([True, True, True, True], device='cuda:0')


In [112]:
_eigval.ndim

1

In [107]:
eigval.ndim

2

In [98]:
entropy

tensor([2.5126, 2.5617, 2.5521, 2.5437], device='cuda:0')

In [13]:
a = torch.rand(4, 32, 128)
_x = a.flatten(2)
pairwise_difference = (torch.unsqueeze(_x,2) - torch.unsqueeze(_x,1))**2
distance = torch.sum(pairwise_difference, dim=3)

In [4]:
_x = a.flatten(2)
test = []
for i in range(len(_x)):
    _x = a[i].flatten(1)
    pairwise_difference = (torch.unsqueeze(_x,1) - torch.unsqueeze(_x,0))**2
    _distance = torch.sum(pairwise_difference, dim=2)
    test.append(_distance)


In [5]:
torch.all(torch.isclose(torch.stack(test), distance))

tensor(True)

In [6]:
_x = a[0].flatten(1)
RBF(a[0].flatten(1), sigma=5, axis=0)


tensor([[1.0000, 0.6446, 0.6807,  ..., 0.6551, 0.6759, 0.6175],
        [0.6446, 1.0000, 0.6532,  ..., 0.6817, 0.7006, 0.6175],
        [0.6807, 0.6532, 1.0000,  ..., 0.6643, 0.6375, 0.6440],
        ...,
        [0.6551, 0.6817, 0.6643,  ..., 1.0000, 0.6694, 0.6276],
        [0.6759, 0.7006, 0.6375,  ..., 0.6694, 1.0000, 0.6789],
        [0.6175, 0.6175, 0.6440,  ..., 0.6276, 0.6789, 1.0000]])

In [7]:
for i in range(len(a)):
    print(torch.all(RBF(a[1].flatten(1), sigma=5, axis=0) == RBF(a.flatten(2), sigma=5, axis=1)[1]))


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [11]:
a = torch.rand(1,4, 32, 128)
RBF(a[0].flatten(1), sigma=5, axis=1)

tensor([[1.0000e+00, 1.0841e-06, 1.0598e-06, 8.3019e-07],
        [1.0841e-06, 1.0000e+00, 1.6050e-06, 9.3722e-07],
        [1.0598e-06, 1.6050e-06, 1.0000e+00, 1.0424e-06],
        [8.3019e-07, 9.3722e-07, 1.0424e-06, 1.0000e+00]])

In [ ]:
result = RBF(a.flatten(2), sigma=5, axis=1)

In [ ]:
result
c, _ = torch.linalg.eigh(result)
a.shape

In [ ]:
eigv = []
for i in range(len(a)):
    _result = RBF(a[i].flatten(1), sigma=5, axis=0)
    e, _ = torch.linalg.eigh(_result)
    print(e.shape)
    eigv.append(e)

In [ ]:
torch.stack(eigv).shape

In [ ]:
torch.all(torch.isclose(torch.stack(eigv), c))

In [ ]:
def entropy2(A: Tensor) -> float:
    eigval, _ = torch.linalg.eigh(A)        
    epsilon = 1e-8
    eigval = eigval.abs() + epsilon 
    return -torch.sum(eigval*(torch.log2(eigval)))

In [ ]:
a[i].flatten(1)

In [ ]:
a.shape